# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Features for Pricing Options
In the previous units, we have taken you through a few features that can be used to predict options prices. You have also learned how to source this data. Once you have all the data you need to transform it before training the ML model. In this notebook, we are going to select the features, transform them and store them in a single dataframe.

The notebook is structured as follows:
1. [Import the Libraries](#import)
2. [Get the Data](#get)
3. [Feature Engineering](#prep)
4. [Conclusion](#conclusion)

<a id='import'></a>
## Import Libraries
We are importing the `pandas` and `numpy` libraries for data manipulation and `warnings` to ignore non-critical warnings.

In [1]:
# For data manipulation
import pandas as pd
import numpy as np

# For ignoring warnings
import warnings
warnings.filterwarnings('ignore')

<a id='get'></a>
## Get the Data
According to the features that we discussed in the previous units, we need three sets of data here, i.e., options chain data, interest rate data, and underlying asset data. We will be predicting the SPX call options prices, so the options data would be pertaining to SPX options and the underlying asset here is `S&P500`.

We have already stored the end-of-month expiry options chain data in a pickle file named `spx_eom_expiry_options_2010_2022.bz2`. We will now import this data using the `read_pickle` method of `pandas` and store it in a variable named `options_data`. We will also need to set the index as the quote date column.

Next, we need the interest data. We will be considering the 1 year US treasury yield as the risk-free rate of return. This data is publicly available and can be sourced from a credible data vendor. We have stored this data in a CSV file named `1_year_treasury_rate_yield.csv` and will import the data using the `read_csv` method of `pandas`.

Lastly, we need the underlying data which is stored in a CSV file named `sp500_index.csv`.

All of the files imported here are available in the zip file of the unit 'Python Codes and Data' in the 'Course Summary' section.

In [2]:
# EOM expiry options data
options_data = pd.read_pickle(
    '../data_modules/spx_eom_expiry_options_2010_2022.bz2')

# Import the interest data
risk_free_rate = pd.read_csv(
    '../data_modules/1_year_treasury_rate_yield.csv', index_col=0)

# Convert index to name and format to datetime
risk_free_rate.index = pd.to_datetime(risk_free_rate.index)
risk_free_rate.columns = ['risk_free_rate']

# Import the underlying data
underlying_data = pd.read_csv(
    '../data_modules/sp500_index_2010_2022.csv', index_col=0)

# Convert index to datetime
underlying_data.index = pd.to_datetime(underlying_data.index)

# Display the first 5 columns
options_data.head()

,[STRIKE],[STRIKE_DISTANCE_PCT],[C_LAST],[UNDERLYING_LAST],[P_LAST],[EXPIRE_DATE],[DTE],[C_DELTA],[C_GAMMA],[C_VEGA],[C_THETA],[C_RHO],[C_IV],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV]
[QUOTE_DATE],,,,,,,,,,,,,,,,,,,
2010-03-01,400.0,0.641,0.0,1115.56,0.45,2010-03-31,29.96,1.0,0.0,0.0,0.0,0.0,,-0.00028,0.00000,0.00309,-0.00486,-0.00056,1.063180
2010-03-01,450.0,0.597,0.0,1115.56,0.75,2010-03-31,29.96,1.0,0.0,0.0,0.0,0.0,,-0.00041,0.00000,0.00408,-0.00476,0.00000,0.944350
2010-03-01,500.0,0.552,0.0,1115.56,0.10,2010-03-31,29.96,1.0,0.0,0.0,0.0,0.0,,0.00000,0.00000,0.00444,-0.00517,-0.00022,0.839210
2010-03-01,550.0,0.507,0.0,1115.56,0.05,2010-03-31,29.96,1.0,0.0,0.0,0.0,0.0,,-0.00025,-0.00002,0.00465,-0.00487,-0.00043,0.743710
2010-03-01,600.0,0.462,186.0,1115.56,0.05,2010-03-31,29.96,1.0,0.0,0.0,0.0,0.0,,-0.00049,0.00002,0.00526,-0.00474,-0.00066,0.655560


<a id='prep'></a>
## Feature Engineering
To predict the options prices we are going to use the same input variables or features as the ones used in most of the parametric models.

In other words, our features will comprise of:<br>
1. Last Traded Price of the Underlying Asset
2. Strike Price
3. Risk-free Rate
4. Days to Expiry (DTE)
5. Volatility

We have the first four inputs but volatility is something that has to be calculated. So let's start by calculating volatility. Here we need to compute the lognormal returns of the underlying asset. The lognormal returns will then be used to calculate its volatility for which we will be using the `rolling` and `std` method. A rolling window of 60 has been selected arbitrarily and we are multiplying it by the square root of 252 `(252**0.5)` to get the annualised volatility.

In [3]:
# Compute the lognormal returns
underlying_data['Log_returns'] = np.log(
    underlying_data['Close']/underlying_data['Close'].shift())

# Compute the volatility and round it up to 2 decimals
underlying_data['volatility'] = round((underlying_data['Log_returns'].rolling(
    60).std()*252**.5), 2)

# Display the last 5 columns
underlying_data.tail()

,Open,High,Low,Close,Volume,Log_returns,volatility
Date,,,,,,,
2022-09-23,3727.139893,3727.139893,3647.469971,3693.229980,5144270000,-0.017383,0.21
2022-09-26,3682.719971,3715.669922,3644.760010,3655.040039,4886140000,-0.010394,0.21
2022-09-27,3686.439941,3717.530029,3623.290039,3647.290039,4577740000,-0.002123,0.21
2022-09-28,3651.939941,3736.739990,3640.610107,3719.040039,4684850000,0.019481,0.21
2022-09-29,3687.010010,3687.010010,3610.399902,3640.469971,4681810000,-0.021353,0.22


The input features will be used to predict the call price of the options. Let's store all this data in a variable named `features_data`.

In [4]:
# Inputs and output
features_data = options_data[[' [STRIKE]', ' [C_LAST]',
                              ' [UNDERLYING_LAST]', ' [DTE]']]

# Change the column names
features_data.columns = ['strike', 'call_price', 'underlying', 'dte']

# Display the last 5 rows
features_data.tail()

,strike,call_price,underlying,dte
[QUOTE_DATE],,,,
2022-09-30,6300.0,0.03,3589.7,0.0
2022-09-30,6400.0,0.00,3589.7,0.0
2022-09-30,6500.0,0.00,3589.7,0.0
2022-09-30,6600.0,0.05,3589.7,0.0
2022-09-30,6700.0,0.05,3589.7,0.0


We need to ensure that the dataset is ready to be used by the machine learning model. Let's go over each column one by one.

1. **Days to Expiry:** Currently, we have the days to expiry but we are converting this into years to expiry. We are doing this to match the unit of `dte` with the unit of other inputs such as `volatility` and `risk_free_rate`.<br>Note: The practise of taking the time to expiry in terms of years is also followed in the Black-Scholes Model.
2. **Risk-free Rate:** We have imported this data but it's not a part of `features_data` yet. Therefore, we will merge the `risk_free_rate` dataframe with `features_data`. Keep in mind that we need to align the risk-free rate with the `features_data` on the basis of date.
3. **Volatility:** The same way as we added the risk-free rate, we will also merge the `volatility` column with `features_data` on the basis of date.

In [5]:
# DTE in terms of years
features_data['years_to_expiry'] = features_data['dte']/365

# Merge the data
features_data = pd.merge(features_data, risk_free_rate,
                         left_index=True, right_index=True)
features_data = pd.merge(features_data, underlying_data['volatility'],
                         left_index=True, right_index=True)

# Display the last 5 rows of the merged data
features_data.tail()

,strike,call_price,underlying,dte,years_to_expiry,risk_free_rate,volatility
2022-09-29,6300.0,0.03,3641.71,1.0,0.00274,3.98,0.22
2022-09-29,6400.0,0.00,3641.71,1.0,0.00274,3.98,0.22
2022-09-29,6500.0,0.00,3641.71,1.0,0.00274,3.98,0.22
2022-09-29,6600.0,0.05,3641.71,1.0,0.00274,3.98,0.22
2022-09-29,6700.0,0.05,3641.71,1.0,0.00274,3.98,0.22


4. **Moneyness:** We will subtract the strike price from the underlying price, which is nothing but moneyness. We are calculating the moneyness for each contract and storing it in a new column `moneyness`.

In [6]:
# Calculate moneyness
features_data['moneyness'] = features_data['underlying'] - \
    features_data['strike']

# Display the last 5 rows
features_data.tail()

,strike,call_price,underlying,dte,years_to_expiry,risk_free_rate,volatility,moneyness
2022-09-29,6300.0,0.03,3641.71,1.0,0.00274,3.98,0.22,-2658.29
2022-09-29,6400.0,0.00,3641.71,1.0,0.00274,3.98,0.22,-2758.29
2022-09-29,6500.0,0.00,3641.71,1.0,0.00274,3.98,0.22,-2858.29
2022-09-29,6600.0,0.05,3641.71,1.0,0.00274,3.98,0.22,-2958.29
2022-09-29,6700.0,0.05,3641.71,1.0,0.00274,3.98,0.22,-3058.29


Some of the options contracts have 0 DTE and some have a call price of 0. So let's select only those contracts where the call price and DTE are not 0.

After adding the moneyness data to our dataset, we can drop the `strike` and `underlying` columns as we should avoid correlated data to increase the accuracy of the model.

Finally, we will drop all rows with NaN values using the `drop_na` method.

In [7]:
# Select only that call price where value is not 0
features_data = features_data[features_data.call_price != 0]

# Select only that DTE where value is not 0
features_data = features_data[features_data.years_to_expiry != 0]

# Avoid correlated data, drop strike and underlying as it has already been used to compute moneyness
features_data = features_data.drop(['strike', 'underlying', 'dte'], axis=1)

# Drop NaN values
features_data = features_data.dropna()

# Display the last 5 rows
features_data.tail()

,call_price,years_to_expiry,risk_free_rate,volatility,moneyness
2022-09-29,0.05,0.00274,3.98,0.22,-2358.29
2022-09-29,0.04,0.00274,3.98,0.22,-2558.29
2022-09-29,0.03,0.00274,3.98,0.22,-2658.29
2022-09-29,0.05,0.00274,3.98,0.22,-2958.29
2022-09-29,0.05,0.00274,3.98,0.22,-3058.29


The accuracy of the ML models reduces as we move away from at the money (ATM) contracts, hence we are selecting the contracts with moneyness between -100 to 100. This can be expanded later on with more complex features and a complex model.

In [8]:
# Filtering the data to reduce the size
features_data = features_data[(
    features_data.moneyness > -100) & ((features_data.moneyness < 100))]

Let's see what the final dataframe looks like.

In [9]:
# Display the last 5 rows of features_data
features_data.tail()

,call_price,years_to_expiry,risk_free_rate,volatility,moneyness
2022-09-29,5.00,0.00274,3.98,0.22,-78.29
2022-09-29,4.28,0.00274,3.98,0.22,-83.29
2022-09-29,3.60,0.00274,3.98,0.22,-88.29
2022-09-29,3.10,0.00274,3.98,0.22,-93.29
2022-09-29,2.51,0.00274,3.98,0.22,-98.29


<a id='conclusion'></a>
## Conclusion
In this notebook, you have learned how to manipulate and transform the features dataset. In the upcoming notebook, we will use this dataset to build a model that will predict options prices.
<br><br>